<img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/><br><br><br><br>



<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# Notebook: NISAR ATBD for Classification of Wetland Inundation Extent
## NASA ISRO Synthetic Aperture Radar Mission
### Combined Algorithm Theoretical Basis Document and Jupyter Notebook for <br> *Classification of Wetland Inundation Extent*

Authors: Bruce Chapman, Paul Siqueira

Date: February 15, 2022 Cloud Best Practices and Gotchas

Modified by Alex Lewandowski, 2024-10-04

*Changes made for the NISAR Early Adopters Workshop, October 2024*

### Summary
This notebook describes the ATBD for generating a wetland inundation product from NISAR time series data stacks. First, the images of the multi-temporal sequence must be well radiometrically calibrated relative to each other, to a higher precision than perhaps required through routine standard calibration of the NISAR imagery. This optional calibration step examines distributed targets that are expected to be unchanged or minimally changed in brightness over a set time span of  an image sequence. With NISAR’s 240 km swath width, it is reasonably assumed that a statistically large area, A<sub>ni</sub>, will not be inundated (or otherwise changing) during any of the 2n observations surrounding the image to be calibrated and classified. These areas will be identified through use of a priori wetlands mask and partly through image segmentation or other methods over the 2n images. <br>
A set of classes will be identified from a multitemporal average of a subset of images including:

- Inundated vegetation (presumption: dominated by double bounce scatter in HH channel) 
- Open water (presumption: low specular scattering in both channels)
- Not inundated (presumption: brighter specular scatter, volume scattering)
- Not classified (presumption: pixels do not align with the scattering model, or no data)

These classes are selected based on calibrated threshold values for the radar backscatter and other metrics. In addition, this same multi-temporal image sequence allows the algorithm to include a more sensitive change detection component for improved robustness. Change detection will allow for refinement within the multitemporal image sequence for change of class during the image sequence that may be more robust than simply classifying the image backscatter and backscatter ratio values.  

Still to be implemented in notebook form:  

- Incorporating datatakes from both ascending and descending orbits with varying incidence angles<br>
- Incorporating datatakes that only have HH channel (no HV)<br>
- Aggregating results to 1 ha<br>
- Evalation against reference data set<br>
- Calibration of thresholds with reference data set<br>



## `git` is the best!

<a id="TOC"></a>
### Table of Contents
0. [Getting Started](#SEC_0)
    <br>0.1 [Import Packages](#SEC_0.1)
    <br>0.2 [Set the Input Parameters](#SEC_0.2)
1. [Get Data](#SEC_1)
    <br>1.1  [Pre-Launch: Get GCOV data from S3 Buckets](#SEC_1.1)
    <br>1.2  [Post-Launch: Query EarthData](#SEC_1.2)
    <br>1.3  [Post-Launch: Query ASF](#SEC_1.3)
    <br>1.4  [Select Images to Include in Time-Series Stack](#SEC_1.4)
    <br>1.5  [Get Geocoding Information from First/Reference Images](#SEC_1.5)
2.  [Process Imagery](#SEC_2)
    <br>2.1  [Read in imagery, with option to combine with band B](#SEC_2.1)
    <br>2.2  [Option: spatial rolling average ](#SEC_2.2)
    <br>2.3  [Option: temporal rolling average of a short time sequence of imagery](#SEC_2.3)
    4.  [Option: calculate correction factors to insure images are relatively calibrated to one another overall](#SEC2.4)
3.  [Classification](#SEC_3)
    <br>3.1  [Set thresholds and classify mean image](#SEC_3.1)
    <br>3.2  [Classify Time Series](#SEC_3.2)
    <br>3.3  [Examine change relative to means for each time stamp to refine classes](#SEC_3.3)
4. [Output Results](#SEC_4)

<a id="SEC_0"></a>
<a id="SEC_0.1"></a>
# 0  &emsp; Getting Started
## 0.1 &emsp; Import Packages


In [ ]:
import os
import glob
from pathlib import Path
import warnings
import re
import datetime
import shutil
import json
from getpass import getpass

import rasterio
from rasterio.plot import show_hist
import struct
import numpy as np
from scipy import signal
import geopandas as gpd
import h5py
from pyproj import CRS

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import colors

import asf_search as asf
import boto3
import earthaccess
import s3fs

<a id="SEC_1"></a>
# 1 &emsp; Get Data


<a id="SEC_1.1"></a>
# 1.1  &emsp; Pre-Launch: Get GCOV Data from S3 Buckets


In [ ]:
from pathlib import Path

aoi = 'Missippi_Vicksburg_MS'
aoi_dir = Path.cwd() / aoi 
atbd_dir = aoi_dir / 'wetlands' # directory doesn't seem to be used
GCOV_dir = aoi_dir / 'GCOV'
TMP_dir = aoi_dir / 'TMP' # directory doesn't seem to be used

for path in [aoi_dir, atbd_dir, GCOV_dir, TMP_dir]:
    path.mkdir(exist_ok=True)

In [ ]:
import boto3

s3 = boto3.resource('s3')

bucket_name = "nisar-calval-public-west2"
bucket_name = "asf-jupyter-data-west"

prefix = 'wetlands/NISAR'

bucket = s3.Bucket(bucket_name)
for obj in bucket.objects.filter(Prefix=prefix, RequestPayer='requester'):
    print(f's3://{bucket_name}/{obj.key}')

In [ ]:
for obj in bucket.objects.filter(Prefix=prefix, RequestPayer='requester'):
    bucket.download_file(obj.key, GCOV_dir/obj.key.split('/')[-1], ExtraArgs={'RequestPayer': 'requester'})
list(GCOV_dir.iterdir())

<a id="SEC_1.2"></a>
# 1.2  &emsp; Post-Launch: Query EarthData with **earthaccess** for GCOV Data 


In [ ]:
# auth = earthaccess.login()

# box_left, box_top, box_right, box_bottom = extent.total_bounds

In [ ]:
# nisar_results = earthaccess.search_data(short_name = 'NISAR_L2_GCOV_BETA_V1', 
#                                         # temporal = ('2023-07-01 00:00:00', '2023-08-31 23:59:59'), # can also specify by time
#                                         # granule_name = '*T00888*', # here we filter by files with CRID value of *T00888*
#                                         bounding_box = (box_left, box_top, box_right, box_bottom ))


In [ ]:
# download_files = earthaccess.download(nisar_results, GCOV_dir) 

<a id="SEC_1.3"></a>
# 1.3  &emsp; Post-Launch: Query ASF with **asf_search** for GCOV Data 


In [ ]:
# Session = asf.ASFSession()
# Session.auth_with_token(getpass("EDL Token:"))


In [ ]:
# opts = asf.ASFSearchOptions(
#     dataset=asf.DATASET.NISAR,
#     session=Session, 
#     )
# gdf = json.loads(open('%s/extent.geojson' %(aoi_dir)).read())
# wkt = str('Polygon((')
# points = len(gdf['features'][0]['geometry']['coordinates'][0][0])
# for i in range(points):
#     wkt = wkt + str(gdf['features'][0]['geometry']['coordinates'][0][0][i][0]) + ' ' + str(gdf['features'][0]['geometry']['coordinates'][0][0][i][1]) 
#     if i<points-1:
#         wkt = wkt + ','
# wkt = wkt + '))'
# results = asf.geo_search(intersectsWith=wkt, opts=opts)

In [ ]:

# list_of_NISAR_GCOVs = [results.geojson()['features'][x]['properties']['url'] for x in range(len(results.geojson()['features']))]
# # print('\n'.join(list_of_NISAR_GCOVs))

# frames = list([f.split('/')[-1].split('_')[7] for f in list_of_NISAR_GCOVs])
# uni_frames = list(np.unique([f.split('/')[-1].split('_')[7] for f in list_of_NISAR_GCOVs]))
# print('\n'.join(uni_frames))


In [ ]:
# import fsspec

# s3_path = 's3://sds-n-cumulus-prod-nisar-products/NISAR_L2_GCOV_BETA_V1/NISAR_L2_PR_GCOV_001_001_A_150_2000_SHNA_A_20231023T060152_20231023T060202_T00787_M_F_J_787/NISAR_L2_PR_GCOV_001_001_A_150_2000_SHNA_A_20231023T060152_20231023T060202_T00787_M_F_J_787.h5'

# s3f = fsspec.open(s3_path, mode='rb', anon=True, default_fill_cache=False)
# h5f = h5py.File(s3f.open(), mode='r')

In [ ]:
# frame = input('which frame do you want to process? %s' %(uni_frames))
# # list_of_NISAR_GCOVs = [k for k in all_results if frame in k]
# indices = [frames.index(k) for k in frames if frame in k]
# # [list_of_NISAR_GCOVs[i] for i in indices]


In [ ]:
# NISAR_ids = []
# response = s3.list_objects_v2(
#             Bucket=bucket_name,
#             Prefix = s3_path)
#             # MaxKeys=100)
#             # Delimiter = '/')
# contents = response.get('Contents')
# existing_NISAR_hdf5 = [contents[i].get('Key').split('/')[-1] for i in range(len(contents)) if '.h5' in contents[i].get('Key')]
# existing_NISAR_hdf5paths = [contents[i].get('Key') for i in range(len(contents)) if '.h5' in contents[i].get('Key')]

# for i in indices:
#     filename =  list_of_NISAR_GCOVs[i].split('/')[-1]
#     NISAR_ids.append(filename)
#     print('Requested File: ', filename)
#     if os.path.isdir(GCOV_dir/filename.split('/')[-1][:-4])==True & (filename in existing_NISAR_hdf5)==True:
#         print('NISAR file is stored on S3 and already available locally')
#     elif (os.path.isfile(GCOV_dir/filename)==False) & (filename in existing_NISAR_hdf5):
#         i = existing_NISAR_hdf5.index(filename)
#         s3_path_new = existing_NISAR_hdf5paths[i]
#         print('\tNISAR HDF5 is already available at S3 PATH: ', s3_path_new)
#         print('\tMove NISAR HDF5 from S3 to local')
#         s3.download_file(bucket_name, s3_path_new , GCOV_dir/filename)
        
#     elif (os.path.isfile(GCOV_dir/filename)==False) & (filename not in existing_NISAR_hdf5):
#         print('\tNISAR HDF5 is not available anywhere')
#         print('\tDownloading NISAR HDF5')
#         try:
#             results[i].download(path=GCOV_dir)#, session=session)
#         except:
#             print('Your .netrc file is not configured. We will authenticate a session with your username and password')
#             user = input('What is your earthdata username?')
#             psw = input('What is your earthdata password?')
#             # session = asf.ASFSession().auth_with_creds(user, pwd)
#             results[i].download(path=GCOV_dir, session=Session)
#         print('\tMoving a copy NISAR HDF5 to S3 bucket')
#         s3.upload_file(Filename= str(GCOV_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(s3_path,aoi,filename))

#     elif (os.path.isfile(GCOV_dir/filename)==True) & (filename not in existing_NISAR_hdf5):
#         print('\tNISAR HDF5 is available locally, but not on S3')
#         print('\tMoving a copy NISAR HDF5 to S3 bucket')
#         s3.upload_file(Filename= str(GCOV_dir / filename), Bucket=bucket_name, Key='%s%s/%s' %(s3_path,aoi,filename))

#     else: 
#         print('\tNISAR HDF5 file exists locally and on S3')
  
    


<a id="SEC_1.4"></a>
# 1.4  &emsp; Select Images to Include in Time-Series Stack


In [ ]:
all_SAR_data = sorted(glob.glob(str(GCOV_dir / '*.h5')))

print('\n'.join([i.split('/')[-1] for i in all_SAR_data]))


In [ ]:
indices = range(0,len(all_SAR_data))
## If you don't want to use all of the images, choose which indices to use now. 
indices = range(0,3)


num=indices[0]
print("Dates of observation (HH and HV):")
date_array = []
SAR_images = []
for ii in indices:
    datestr = all_SAR_data[ii].split('/')[-1].split('_')[11][:8]
   
    date_obj = datetime.datetime.strptime(datestr,'%Y%m%d')
    print(ii, '\t',date_obj)
    date_array.append(date_obj)
    SAR_images.append(all_SAR_data[ii])


<a id="SEC_1.3"></a>
# 1.5  &emsp; Get Geocoding Information from First/Reference Images


In [ ]:
ref_image = all_SAR_data[0]
f = h5py.File(ref_image, "r") 
a_group_key = list(f.keys())[0]
ds_x = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['xCoordinates'][()]      # returns as a h5py dataset object
ds_y = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['yCoordinates'][()]      # returns as a h5py dataset object
ds_epsg = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['projection'][()]
cols = ds_x.shape[0]
rows = ds_y.shape[0]
print('X Size: ',cols,' Y Size: ',rows)

yres = abs(ds_y[0] - ds_y[1])
xres = abs(ds_x[0] - ds_x[1])
print('Resolution X:', xres, ' Y:',yres,'m')

# ref_te = ds_x[0],ds_x[-1],ds_y[0],ds_y[-1]
ulx = xres * round((ds_x[0] - ((ds_x[1] - ds_x[0])/2))/xres)
lrx = xres * round(ds_x[-1]/xres)
uly = yres * round((ds_y[0] - ((ds_y[1] - ds_y[0])/2))/yres)
lry = yres * round(ds_y[-1]/yres)
print('Raster bounds: ',ulx,lrx,uly,lry)#min(ds_x),max(ds_x),min(ds_y),max(ds_y))
crop = False
if crop:
    # crop = gpd.read_file(aoi_dir / 'crop.geojson')
    ulx,lry,lrx,uly = 664836.2728999999817461,3528994.8253000001423061,690193.6014999999897555,3560044.6154000000096858
    print('Crop bounds: ', ulx,lrx,uly,lrx)
geotransform = [ulx, xres, 0.0, uly, 0.0, -yres]

spatialref = CRS.from_user_input(ds_epsg).to_wkt()
print(ds_epsg)

In [ ]:
tmp_lon = np.meshgrid(ds_x,ds_y)[0]
tmp_lat = np.meshgrid(ds_x,ds_y)[1]
crop_xy = np.where((tmp_lon>ulx) & (tmp_lon <lrx) & (tmp_lat >lry) & (tmp_lat <uly),1,0)
coords = np.argwhere(crop_xy)
x_min, y_min = coords.min(axis=0)
x_max, y_max = coords.max(axis=0)
cropped = crop_xy[x_min:x_max+1, y_min:y_max+1]

cols = cropped.shape[1]
rows = cropped.shape[0]

## Water Level from nearby USGS water level gage with above image acquisition dates, showing the change in water level during the course of those acquisitions.

https://rivergages.mvr.usace.army.mil/WaterControl/stationinfo2.cfm?sid=CE40FF58&fid=VCKM6&dt=S

<img style="float: left;" src="images/water_level_chart.png"><br>


<a id="SEC_2"></a>
<a id="SEC_2.1"></a>
# 2 &emsp; Process Imagery
# 2.1  &emsp; Read in imagery, with option to combine with band B

In [ ]:
# mask = np.ones((rows,cols)) 

# Read raster files and make them into a 3D numpy array
HH = []
HV = []

for image in SAR_images:
    print(image.split('/')[-1])
    f = h5py.File(image, "r") 
    a_group_key = list(f.keys())[0]
    ds_HH = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HHHH'][()] 
    ds_HV = f[a_group_key]['LSAR']['GCOV']['grids']['frequencyA']['HVHV'][()] 
    ds_HH = ds_HH[x_min:x_max+1, y_min:y_max+1]
    ds_HV = ds_HV[x_min:x_max+1, y_min:y_max+1]
    if (ds_HH.shape[1] == cols) & (ds_HH.shape[0] == rows):
        ftemp = np.clip(ds_HH,0.0,10000.0)
        HH.append(ftemp)
        ftemp = np.clip(ds_HV,0.0,10000.0)
        HV.append(ftemp)
    else:
        print('Dimensions of this SAR image do not match the reference image, SKIP')
rasterstack_HH = np.array(HH, dtype=float)
rasterstack_HV = np.array(HV, dtype=float)

#option: combine band A and band B by multilooking
#for some reason, band B is in the opposite order
# for ii in range(len(rasterstack_HH)):
#     rasterstack_HH[ii,:,:]=(rasterstack_HH[ii,:,:]+rasterstack_HH_B[num-ii,:,:])/2.0
#     rasterstack_HV[ii,:,:]=(rasterstack_HV[ii,:,:]+rasterstack_HV_B[num-ii,:,:])/2.0
    
#option: Classify band B
#for some reason, band B is in the opposite order
#for ii in range(len(rasterstack_HH)):
#    rasterstack_HH[ii,:,:]=rasterstack_HH_B[num-ii,:,:]
#    rasterstack_HV[ii,:,:]=rasterstack_HV_B[num-ii,:,:]
    
ss=np.zeros(3,dtype=np.int16)
ss[0]=int(rasterstack_HV.shape[0])
ss[1]=int(rasterstack_HV.shape[1])
ss[2]=int(rasterstack_HV.shape[2])


<a id="SEC_2.2"></a>
# 2.2  &emsp; Option: spatial rolling average 

In [ ]:
#5x5
#ispat=0 - no spatial averaging
#ispat=1 - execute spatial averaging comparable to output pixel spacing (ie about 5x5)
#  -- but UAVSAR pixel spacing is just 6 m, so this correponds to an output pixel spacing of 30 m.
ispat=1
if ispat == 1:
    ks = 5
    kernel=(ks,ks)
    kernel = np.ones(kernel) /(ks*ks)
    for kk in range(ss[0]):
        rasterstack_HH[kk,:,:] = signal.convolve2d(rasterstack_HH[kk,:,:], kernel, mode='same')
        rasterstack_HV[kk,:,:] = signal.convolve2d(rasterstack_HV[kk,:,:], kernel, mode='same')
 

<a id="SEC_2.3"></a>
# 2.3  &emsp; Option: temporal rolling average of a short time sequence of imagery

In [ ]:
#if temporal rolling average is not desired, set Nave_rolling =1 below
#number of images in temporal rolling average
Nave_rolling = 1

np.seterr(divide='ignore')  # python doesn't like dividing zero by zero, NaN by NaN, etc.
np.seterr(invalid='ignore')

ss = rasterstack_HH.shape
#number of rolling averages
Nimages = ss[0]- Nave_rolling + 1
output_stack_dim = (Nimages,ss[1],ss[2])   # standard size of output rasterstack

# define arrays, filled with zeros
rasterstack_rolling_HH = np.zeros(output_stack_dim)

#if dual pol data. Otherwise, will have to just use HH for classification.

rasterstack_rolling_HV = np.zeros(output_stack_dim)
rasterstack_rolling_ratio = np.zeros(output_stack_dim)
#options:  product of HH and HV, or Sum of HH and HV
#here, using product.
rasterstack_rolling_prod = np.zeros(output_stack_dim)

for ii in range(len(rasterstack_HH)):

    for jj in range(-Nave_rolling+1,1):  # example:  -1 to 1
        kk = ii + jj
        if(kk>=0 and kk<Nimages):
            # print(ii,jj,kk)   # ii - input image, kk - output image
            # note that the rolling average for image 0 contains images 0, 1, etc., up until the number of averages,
            #      but no earlier than image 0
            rasterstack_rolling_HH[kk,:,:] = rasterstack_rolling_HH[kk,:,:]+rasterstack_HH[ii,:,:]
            rasterstack_rolling_HV[kk,:,:] = rasterstack_rolling_HV[kk,:,:]+rasterstack_HV[ii,:,:]
            rasterstack_rolling_ratio[kk,:,:] = np.divide(rasterstack_rolling_HH[kk,:,:],rasterstack_rolling_HV[kk,:,:])+np.divide(rasterstack_HH[ii,:,:],rasterstack_HV[ii,:,:])
            rasterstack_rolling_prod[kk,:,:] = np.multiply(rasterstack_rolling_HH[kk,:,:],rasterstack_rolling_HV[kk,:,:])+np.multiply(rasterstack_HH[ii,:,:],rasterstack_HV[ii,:,:])

rasterstack_rolling_HH[:,:,:]=rasterstack_rolling_HH[:,:,:]/Nave_rolling
rasterstack_rolling_HV[:,:,:]=rasterstack_rolling_HV[:,:,:]/Nave_rolling
rasterstack_rolling_ratio[:,:,:]=rasterstack_rolling_ratio[:,:,:]/Nave_rolling
rasterstack_rolling_prod[:,:,:]=rasterstack_rolling_prod[:,:,:]/Nave_rolling


<a id="SEC_2.4"></a>
# 2.4  &emsp; Option: calculate correction factors to insure images are relatively calibrated to one another overall

<bold>TODO: optionally select region to calculate normalization factor</bold>

In [ ]:
#assumes minor over brightness fluctuations
#could be refined to not include veg. inundated areas or areas that are changing significantly
#could apply HV correction factor to HH and HV (not affected by inundation, assume similar trends)
#the original input arrays are not calibrated

#To do:  implement method to apply to image subset only, using a shapefile or pixel coordinates

np.seterr(invalid='ignore')

# options for normalization:
# ical=0 - don't normalize
# ical=1 normalize both HH and HV by HV correction factor
# ical=2 - normalize each channel separately and individually
ical=1

#define arrays
rasterstack_rolling_corrected_HH = np.zeros(output_stack_dim)
rasterstack_rolling_corrected_HV = np.zeros(output_stack_dim)
rasterstack_rolling_corrected_ratio = np.zeros(rasterstack_rolling_corrected_HH.shape)
rasterstack_rolling_corrected_product = np.zeros(rasterstack_rolling_corrected_HH.shape)

#zeros masked out previously
mean_HH = np.nanmean(rasterstack_HH)
mean_HV = np.nanmean(rasterstack_HV)

#print(mean_HH)
#print(mean_HV)

correction_factor_HH = np.zeros(Nimages)
correction_factor_HV = np.zeros(Nimages)
print("Correction factors for each image:")
print(" ")

print('  #    HHfac    HVfac')
print(' ---   -----    -----')
for ii in range(len(rasterstack_rolling_HH)):
    correction_factor_HH[ii] = np.nanmean(rasterstack_rolling_HH[ii,:,:])/mean_HH
    correction_factor_HV[ii] = np.nanmean(rasterstack_rolling_HV[ii,:,:])/mean_HV
    if ical == 0:
        correction_factor_HH[ii]=1.
        correction_factor_HV[ii]=1.
    if ical == 1:
        correction_factor_HV[ii]=np.nanmean(rasterstack_rolling_HV[ii,:,:])/mean_HV
        correction_factor_HH[ii]=correction_factor_HV[ii]
    if ical == 2:
        correction_factor_HV[ii]=np.nanmean(rasterstack_rolling_HV[ii,:,:])/mean_HV
        correction_factor_HH[ii]=np.nanmean(rasterstack_rolling_HH[ii,:,:])/mean_HH

    print('%3d   %6.3f   %6.3f' % ( ii, correction_factor_HH[ii], correction_factor_HV[ii]) )
    
for kk in range(Nimages):
    rasterstack_rolling_corrected_HH[kk,:,:] = rasterstack_rolling_HH[kk,:,:]/correction_factor_HH[kk]
    rasterstack_rolling_corrected_HV[kk,:,:] = rasterstack_rolling_HV[kk,:,:]/correction_factor_HV[kk]
    rasterstack_rolling_corrected_ratio[kk,:,:]=rasterstack_rolling_corrected_HH[kk,:,:]/rasterstack_rolling_corrected_HV[kk,:,:]
    rasterstack_rolling_corrected_product[kk,:,:]=rasterstack_rolling_corrected_HH[kk,:,:]*rasterstack_rolling_corrected_HV[kk,:,:]  

#remove NaNs
rasterstack_rolling_corrected_ratio=np.nan_to_num(rasterstack_rolling_corrected_ratio)

# Display corrected rolling averages, channel ratios and products

In [ ]:
plt.rcParams['figure.figsize'] = (40,80)
for ii in range(Nimages):

    plt.subplot(Nimages,4,ii*4+1)
    plt.imshow(rasterstack_rolling_corrected_HH[ii,:,:],vmin=0.0,vmax=0.7,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("HH avg corr", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)
    plt.subplot(Nimages,4,ii*4+2)
    plt.imshow(rasterstack_rolling_corrected_HV[ii,:,:],vmin=0.0,vmax=0.15,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("HV avg Corr", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)
    plt.subplot(Nimages,4,ii*4+3)
    plt.imshow(rasterstack_rolling_corrected_HH[ii,:,:]/rasterstack_rolling_corrected_HV[ii,:,:],vmin=4,vmax=12,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("Ratio", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)
    plt.subplot(Nimages,4,ii*4+4)
    plt.imshow(rasterstack_rolling_corrected_product[ii,:,:],vmin=0,vmax=.0002,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("Product", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)

# Calculate the mean for each stack of images, and the change from the mean for each time period

In [ ]:
#Still to be implemented:  change relative to prior image

change_HH = np.zeros(output_stack_dim)
change_HV = np.zeros(output_stack_dim)
change_ratio = np.zeros(output_stack_dim)
change_product = np.zeros(output_stack_dim)

rasterstack_HH=np.nan_to_num(rasterstack_HH)
rasterstack_HV=np.nan_to_num(rasterstack_HV)

#check if nans are present
array_sum = np. sum(rasterstack_HH)
array_has_nan = np. isnan(array_sum)
print('Are there any NaNs in HH?', array_has_nan)
array_sum = np. sum(rasterstack_HV)
array_has_nan = np. isnan(array_sum)
print('Are there any NaNs in HV?', array_has_nan)


#disable warning: all numbers in stack at one pixel are zero -> causes warning
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    mean_stack_HH = np.mean(rasterstack_HH, axis=0,where=np.greater(rasterstack_HH,0.))
    mean_stack_HV = np.mean(rasterstack_HV, axis=0,where=np.greater(rasterstack_HV,0.))
    
mean_stack_HH=np.nan_to_num(mean_stack_HH)
mean_stack_HV=np.nan_to_num(mean_stack_HV)
      
mean_stack_HH=np.nan_to_num(mean_stack_HH)
mean_stack_HV=np.nan_to_num(mean_stack_HV)

output_dim = (ss[1],ss[2])  # standard size of output rasterstack

# define arrays, filled with zeros
mean_prod = np.zeros(output_dim)
mean_ratio = np.zeros(output_dim)

mean_prod[:,:]=(mean_stack_HH[:,:]*mean_stack_HV[:,:])
mean_ratio=mean_stack_HH[:,:]/mean_stack_HV[:,:]

for kk in range(Nimages):
#relative to mean
    change_HH[kk,:,:]=rasterstack_rolling_corrected_HH[kk,:,:]/mean_stack_HH
    change_HV[kk,:,:]=rasterstack_rolling_corrected_HV[kk,:,:]/mean_stack_HV
    change_ratio[kk,:,:]=rasterstack_rolling_corrected_ratio[kk,:,:]/mean_ratio
    change_product[kk,:,:]=rasterstack_rolling_corrected_product[kk,:,:]/mean_prod

change_HH=np.nan_to_num(change_HH)
change_HV=np.nan_to_num(change_HV)
change_ratio=np.nan_to_num(change_ratio)
change_product=np.nan_to_num(change_product)

# Display change over time of corrected rolling averages, channel ratios and products

In [ ]:
print("Change relative to mean values")
plt.rcParams['figure.figsize'] = (40,50)
for ii in range(Nimages):
    plt.subplot(Nimages,4,ii*4+1)
    plt.imshow(change_HH[ii,:,:],vmin=0.5,vmax=1.5,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("HH /mean_HH", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)
    plt.subplot(Nimages,4,ii*4+2)
    plt.imshow(change_HV[ii,:,:],vmin=0.5,vmax=1.5,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("HV /mean_HV", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)
    plt.subplot(Nimages,4,ii*4+3)
    plt.imshow(change_ratio[ii,:,:],vmin=0.5,vmax=1.5,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("Ratio / mean ratio", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)
    plt.subplot(Nimages,4,ii*4+4)
    plt.imshow(change_product[ii,:,:],vmin=0.5,vmax=1.5,cmap=plt.cm.gray)
    plt.title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
    plt.title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
    plt.title("Product / mean product", loc="right",fontdict = {'fontsize' : 9})
    plt.colorbar(shrink=0.4)

# Display mean values of HH, HV, products, and ratios

In [ ]:
fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize=(10,10))
cax = ax1.imshow(mean_stack_HH,vmin=0,vmax=0.5)
ax1.set_title('HH')
plt.colorbar(cax,ax=ax1,shrink = 0.3)
cax = ax2.imshow(mean_stack_HV,vmin=0,vmax=0.1)
ax2.set_title('HV')
plt.colorbar(cax,ax=ax2,shrink = 0.3)
cax = ax3.imshow(mean_prod,vmin=0,vmax=0.01)
ax3.set_title('HH*HV')
plt.colorbar(cax,ax=ax3,shrink = 0.3)
cax = ax4.imshow(mean_ratio,vmin=2,vmax=10)
ax4.set_title('HH/HV')
plt.colorbar(cax,ax=ax4,shrink = 0.3)
# plt.imshow(mean_prod,vmin=0,vmax=.01)
# plt.colorbar(shrink=0.1)

<a id="SEC_3"></a>
# 3 &emsp; Classification

If the inundation state is relatively constant over the time span of the imagery, these classes may be a useful reference. However, if the inundation state is variable over the time span, this classification may be biased depending on the classification thresholds and the magnitude of the changes in backscatter.<br>
For classifying open water, HV in many cases is a more reliable indicator that HH.  However, to take advantage of both HH and HV, we find the product and threshold based on the product.<br>
For classifying inundated vegetation, typically we find that double bounce will dominate the HH backscatter and HH will be brighter than expected from the HV backscatter, which is dominated by volume scatter even in the presence of double bounce reflections occuring.  Since HH also gets brighter and darker as biomass increases or decreases, we have found HH/HV to be a valuable metric for identifying inundated vegatation, even for those cases were the double bounce contribution to HH may be smaller in herbaceous areas (for which we assign a separate class).  We therefore apply thresholds based on HH and HH/HV.

# Inundation classes

The basis of the classification will be the simplest possible technique to demonstrate sensitivity to inundation state: identifying threshold values in the imagery that correspond different inundation states. A possible augmention would be to first segment the image values by pixel similarity.  

(not all values currently used)

#0 no data or not classified<br>
#1 - 5 open water<br>
#7 - 9  no surface water<br>
#11 - 13 is inundated veg due to high HH/HV <br>
#15 - 17 is inundated veg with bright HH return<br>



<a id="SEC_3.1"></a>
# 3.1  &emsp; Set thresholds and classify mean image

<bold>TODO: optionally select region to calculate normalization factor</bold>

In [ ]:
np.seterr(invalid='ignore')  # python doesn't like dividing zero by zero, NaN by NaN, etc.

#Thresholds to be calibrated based on other data sets:
# open water class 1
T1_prod_min =  0.000001#0.000001
T1_prod_max =  0.0001#.00005
# inundated veg class 16 - bright HH
T2_HH_min =  0.1 #0.5
T2_HH_max =  0.5#10.
T2_ratio_min= 6 #6
T2_ratio_max=100 #500
# inundated veg class 12  - not as bright HH in class 16, but high HH/HV ratio
T3_HH_min =  0.001#0.1
T3_HH_max =  0.2#0.3
T3_ratio_min= 4 #8
T3_ratio_max=500

classif=np.zeros(output_dim, dtype=np.int8)

#all data
classif=np.where(mean_stack_HH > 0.0,8,classif)
#open water
classif=np.where((mean_prod < T1_prod_max) & (mean_prod > T1_prod_min),3,classif)
#classify inundated veg where not classified as open water
classif=np.where((classif != 3)  & (mean_stack_HH > T2_HH_min) & (mean_stack_HH < T2_HH_max) \
                 &(mean_ratio < T2_ratio_max) & (mean_ratio > T2_ratio_min),16,classif)

# classif=np.where((classif != 3)  & (mean_stack_HH > T3_HH_min) & (mean_stack_HH < T3_HH_max) \
#                  &(mean_ratio < T3_ratio_max) & (mean_ratio > T3_ratio_min),12,classif)


# Display classification of mean image

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,40)

#define color table for this and subsequent plots
colors =["None","blue","blue","blue","blue","blue",'green','green','green','green','yellow','yellow','yellow','yellow',
         'gold','gold','gold','gold']
cmap = ListedColormap(colors)

# plt.subplot(Nimages,1,ii+1)
cax = plt.imshow(classif,vmin=1,vmax=16,cmap=cmap,interpolation='none')
plt.title(date_array[0], loc="left",fontdict = {'fontsize' : 9})
plt.title(date_array[Nimages-1], loc="center",fontdict = {'fontsize' : 9})
plt.title("class", loc="right",fontdict = {'fontsize' : 9})
# plt.colorbar(shrink=0.1)

cbar = fig.colorbar(cax, ticks=[4,8,11,14],shrink=0.1)
cbar.ax.set_yticklabels(['Open Water','No Surface Water','Inundated Vegetation (High HH:HV)','Inundated Vegetation (High HH)'])  # vertically oriented colorbar
                         
                         
                         

<a id="SEC_3.2"></a>
# 3.2  &emsp; Classify Time Series

In [ ]:
#classify time series

#Thresholds to be calibrated based on other data sets:
# open water class 1
T1_prod_min =  0.000001
T1_prod_max =  .00005
# inundated veg class 16 - bright HH
T2_HH_min =  0.5
T2_HH_max =  10.
T2_ratio_min=6
T2_ratio_max=500
# inundated veg class 12  - not as bright HH in class 16, but has high HH/HV ratio
T3_HH_min =  0.1
T3_HH_max =  0.3
T3_ratio_min=8
T3_ratio_max=500

class_stack=np.zeros(output_stack_dim,dtype=np.int8)
change_OW=np.zeros(class_stack.shape[0],dtype=np.int32)
change_FL=np.zeros(class_stack.shape[0],dtype=np.int32)
change_NF=np.zeros(class_stack.shape[0],dtype=np.int32)

for kk in range(len(class_stack)):
    class_stack[kk,:,:]=np.where(rasterstack_rolling_corrected_HH[kk,:,:] > 0.0000001,8,class_stack[kk,:,:])

    class_stack[kk,:,:]=np.where((rasterstack_rolling_corrected_product[kk,:,:] < T1_prod_max) \
                                 & (rasterstack_rolling_corrected_product[kk,:,:] > T1_prod_min),3,class_stack[kk,:,:])

#classify inundated veg where not classified as open water
    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] != 3)  \
                                 & (rasterstack_rolling_corrected_HH[kk,:,:] > T2_HH_min) \
                                 & (rasterstack_rolling_corrected_HH[kk,:,:] < T2_HH_max) \
                                 &(rasterstack_rolling_corrected_ratio[kk,:,:] < T2_ratio_max) \
                                 & (rasterstack_rolling_corrected_ratio[kk,:,:] > T2_ratio_min),16,class_stack[kk,:,:])


    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] != 3)  \
                                  & (rasterstack_rolling_corrected_HH[kk,:,:] > T3_HH_min) \
                                  & (rasterstack_rolling_corrected_HH[kk,:,:] < T3_HH_max) \
                                  &(rasterstack_rolling_corrected_ratio[kk,:,:] < T3_ratio_max) \
                                  & (rasterstack_rolling_corrected_ratio[kk,:,:] > T3_ratio_min),12,class_stack[kk,:,:])

    #track changes
    change_OW[kk]=np.count_nonzero(class_stack[kk,:,:] == 3)
    
    change_FL[kk]=np.count_nonzero(class_stack[kk,:,:] == 12)+np.count_nonzero(class_stack[kk,:,:] == 16)
    
    change_NF[kk]=np.count_nonzero(class_stack[kk,:,:] == 8)

#    print(f"Number of Zeroes in Array --> {class_stack[kk,:,:][np.where( class_stack[kk,:,:]== 0)].size}")


# Display change in inundation vs time

In [ ]:
#sanity check
plt.rcParams['figure.figsize'] = (6,4)
fig = plt.figure()
X = np.arange(class_stack.shape[0])
ax = fig.add_axes([0,0,1,1])
classn=[0,1,2,3,4]
colors = {'Not Inundated':'green', 'Open Water':'blue','Inundated Veg':'yellow'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels)
#ax.legend(labels=['Not Flooded','Open Water', 'Inundated Veg'])
ax.bar(X+0.50,change_NF,color='green',width=0.25)
ax.bar(X+0.00,change_OW,color='blue',width=0.25)
ax.bar(X+0.25,change_FL,color='yellow',width=0.25)
plt.show()

# Display time sequence of classes

In [ ]:
date_array[ii+Nave_rolling-1].strftime("%m/%d/%Y")

In [ ]:
plt.rcParams['figure.figsize'] = (8,32)
import imageio
with imageio.get_writer('%s/inundation_classes.gif' %(aoi_dir),mode='I',duration=300) as writer:
    for ii in range(Nimages):

        plt.subplot(Nimages,1,ii+1)

        plt.imshow(class_stack[ii,:,:],vmin=1,vmax=16,cmap=cmap,interpolation='none')
        plt.title(date_array[ii].strftime("%m/%d/%Y"), loc="left",fontdict = {'fontsize' : 9})
        plt.title(date_array[ii+Nave_rolling-1].strftime("%m/%d/%Y"), loc="center",fontdict = {'fontsize' : 9})
        plt.title("class", loc="right",fontdict = {'fontsize' : 9})
        plt.colorbar(shrink=0.8)

        plt.savefig('%s/%s.png' %(aoi_dir,date_array[ii].strftime("%Y%m%d")))
        image = imageio.v2.imread('%s/%s.png' %(aoi_dir,date_array[ii].strftime("%Y%m%d")))
        writer.append_data(image)

<a id="SEC_3.3"></a>
# 3.3  &emsp; Examine change relative to means for each time stamp to refine classes

Change in HH backscatter and in HH/HV from the mean or the previous HH and HH/HV values can be a more robust indicator of a change occuring in class. Thresholds are applied based on those change metrics.

Not yet implemented option: change focused on changes from typical inundation state derived from the mean imagery over the entire time span. Detect change in backscatter for each time stamp within identified classes from mean imagery to refine classes. This will be an augmentation to the change detection implemented below for all areas and is TBD.

In [ ]:
change_OW_2=np.zeros(class_stack.shape[0],dtype=np.int32)
change_FL_2=np.zeros(class_stack.shape[0],dtype=np.int32)
change_NF_2=np.zeros(class_stack.shape[0],dtype=np.int32)

T5_HH_inc=2
T5_HH_dec=0.5
T5_ratio_inc=2
T5_ratio_dec=0.5

T6_HH_inc=2
T6_HH_dec=0.5
T6_ratio_inc=2
T6_ratio_dec=0.5

T7_HH_inc=4
T7_HH_dec=0.25
T7_ratio_inc=4
T7_ratio_dec=0.25

for kk in range(Nimages):

#open water increased by x, -> no surface water
    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] == 3)  \
                                 & (change_HH[kk,:,:] > T5_HH_inc) \
                                 & (change_ratio[kk,:,:] < T5_ratio_inc),7,class_stack[kk,:,:])
    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] == 3)  \
                                 & (change_HH[kk,:,:] > T5_HH_inc) \
                                 & (rasterstack_rolling_corrected_ratio[kk,:,:] > T3_ratio_max),11,class_stack[kk,:,:])
    
#inundated veg decreased by x, -> no surface water
    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] == 16)  \
                                 & (change_HH[kk,:,:] < T6_HH_dec) \
                                 & (change_ratio[kk,:,:] < T6_ratio_dec),9,class_stack[kk,:,:])
    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] == 12)  \
                                 & (change_HH[kk,:,:] < T6_HH_dec) \
                                 & (change_ratio[kk,:,:] < T6_ratio_dec),9,class_stack[kk,:,:])
     
#no surface water decreased by x, -> open water
    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] == 8)  \
                                 & (change_HH[kk,:,:] < T5_HH_dec) \
                                 & (change_ratio[kk,:,:] > T5_ratio_inc),4,class_stack[kk,:,:])
#no surface water increased by x, -> inundated veg
    class_stack[kk,:,:]=np.where((class_stack[kk,:,:] == 8)  \
                                 & (change_HH[kk,:,:] > T6_HH_inc) \
                                 & (change_ratio[kk,:,:] > T6_ratio_inc),13,class_stack[kk,:,:])

#track changes
    change_OW_2[kk]=np.count_nonzero(class_stack[kk,:,:] == 1)+np.count_nonzero(class_stack[kk,:,:] == 2)\
    +np.count_nonzero(class_stack[kk,:,:] == 3)+np.count_nonzero(class_stack[kk,:,:] == 4)\
    +np.count_nonzero(class_stack[kk,:,:] == 5)

    change_FL_2[kk]=np.count_nonzero(class_stack[kk,:,:] == 11)+np.count_nonzero(class_stack[kk,:,:] == 12)\
    +np.count_nonzero(class_stack[kk,:,:] == 13)\
    +np.count_nonzero(class_stack[kk,:,:] == 15)\
    +np.count_nonzero(class_stack[kk,:,:] == 16)\
    +np.count_nonzero(class_stack[kk,:,:] == 17)\
    
    change_NF_2[kk]=np.count_nonzero(class_stack[kk,:,:] == 7)+np.count_nonzero(class_stack[kk,:,:] == 8)\
    +np.count_nonzero(class_stack[kk,:,:] == 9)
    



# To assess impact of change detection to classification results, plot the change in number of pixels in each class for each time period.

In [ ]:
#display change in inundation vs time

print("Change in surface water classes")
plt.rcParams['figure.figsize'] = (6,4)
fig = plt.figure()
X = np.arange(class_stack.shape[0])
ax = fig.add_axes([0,0,1,1])
classn=[0,1,2,3,4]
colors = {'Not Inundated':'green', 'Open Water':'blue','Inundated Veg':'yellow'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels)
ax.bar(X+0.50,100*(change_NF_2-change_NF),color='green',width=0.25)
ax.bar(X+0.00,100*(change_OW_2-change_OW),color='blue',width=0.25)
ax.bar(X+0.25,100*(change_FL_2-change_FL),color='yellow',width=0.25)

plt.show()

# Display final time sequence of wetland inundation classes

In [ ]:
plt.rcParams['figure.figsize'] = (8,32)
with imageio.get_writer('%s/inundation_classes.gif' %(aoi_dir),mode='I',duration=300) as writer:

    for ii in range(Nimages):

        # plt.subplot(Nimages,1,ii+1)
        fig,ax = plt.subplots(1,1,figsize=(10,10))
        im = ax.imshow(class_stack[ii,:,:],vmin=1,vmax=16,cmap=cmap,interpolation='none')
        ax.set_title(date_array[ii], loc="left",fontdict = {'fontsize' : 9})
        ax.set_title(date_array[ii+Nave_rolling-1], loc="center",fontdict = {'fontsize' : 9})
        ax.set_title("class", loc="right",fontdict = {'fontsize' : 9})
        # plt.colorbar(im,ax=ax,shrink=0.8)
        cbar = fig.colorbar(im, ticks=[4,8,11,14],shrink=0.8)
        cbar.ax.set_yticklabels(['Open Water','No Surface Water','Inundated Vegetation (High HH:HV)','Inundated Vegetation (High HH)'])  # vertically oriented colorbar

        plt.savefig('%s/%s.png' %(aoi_dir,date_array[ii].strftime("%Y%m%d")))
        image = imageio.v2.imread('%s/%s.png' %(aoi_dir,date_array[ii].strftime("%Y%m%d")))
        writer.append_data(image)
        # plt.close()

<a id="SEC_4"></a>
# 4  &emsp; Output results

In [ ]:
#output results
for kk in range(len(class_stack)):
    outname = aoi_dir/ (date_array[kk].strftime("%m_%d_%Y") + '_class.bin')
    fid = open(outname,'wb')
    print(kk,outname)
    tmp = np.byte(np.copy(class_stack[kk,:,:]))
    fid.write(tmp)
    fid.close();


# Plot the area of each class within the imagery for each time period

In [ ]:
#display change in inundation vs time

print("Change in area of surface water classes over time")
plt.rcParams['figure.figsize'] = (6,4)
fig = plt.figure()
X = np.arange(class_stack.shape[0])
ax = fig.add_axes([0,0,1,1])
classn=[0,1,2,3,4]
colors = {'Not Inundated':'green', 'Open Water':'blue','Inundated Veg':'yellow'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels)
ax.bar(X+0.50,change_NF_2,color='green',width=0.25)
ax.bar(X+0.00,change_OW_2,color='blue',width=0.25)
ax.bar(X+0.25,change_FL_2,color='yellow',width=0.25)
plt.savefig(aoi_dir/ 'AreaChange.png')
plt.show()



# Water level gage data at nearby Vicksburg Bridge on the Mississippi River
(water level does not generally correspond linearly to surface water extent)

https://rivergages.mvr.usace.army.mil/WaterControl/stationinfo2.cfm?sid=CE40FF58&fid=VCKM6&dt=S

<img style="float: left;" src="images/water_level_chart1.png">
